# Data Wrangling

## Dataset - WeRateDogs&trade; Twitter Archive

***By: Kartik Nanduri***<br>
**Date: 21st Nov, 2018.**

## Gathering

In [1]:
# importing all the necessary libraries
import os
import pandas as pd
import requests as req

***<span style="color: red">Important! uncomment the following files to run the book with out errors.</span>***

In [ ]:
# resetting the folder structure.
#os.rename('dataset/twitter-archive-enhanced.csv', 'twitter-archive-enhanced.csv')
#import shutil
#shutil.rmtree('dataset')

***<span style="color: green">Important! once done, please recomment.</span>***

1. [x] **The file given at hand `twitter-archive-enhanced.csv`**

In [2]:
# all the requried files for this project are in the list files_list
files_list = ['twitter-archive-enhanced.csv', 'image-predictions.tsv', 'tweet_json.txt']

In [3]:
# reading the twitter archive file
archive = pd.read_csv(files_list[0])

# taking at random entries for the archive file
archive.shape

(2356, 17)

2. [x] **Fetching the data from url and saving it to local drive - `image-predictions.tsv`**

In [4]:
# reading the file from internet using the requests library
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
res = req.get(url)

with open(url.split('/')[-1], mode = "wb") as op_file:
    op_file.write(res.content)

In [5]:
# checking if fetched the data right way
img_pre_test = pd.read_csv(files_list[1], delimiter = "\t", encoding = 'utf-8')
img_pre_test.sample(2)

# we did it the right way, Yay! it worked.

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
903,700062718104104960,https://pbs.twimg.com/media/CbcfUxoUAAAlHGK.jpg,1,hummingbird,0.180998,False,peacock,0.135179,False,eel,0.075371,False
1770,827600520311402496,https://pbs.twimg.com/media/C3w6RYbWQAAEQ25.jpg,1,Pembroke,0.325638,True,golden_retriever,0.317235,True,Labrador_retriever,0.116087,True


3. [x] **Getting data from Twitter&trade;**

In [6]:
# importing all the necessary libraries for accessing Twitter via API
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

In [7]:
# setting up all the necessary placeholders for API
consumer_key = 'xxx.xxx.xxx.xxx'
consumer_secret = 'xxx.xxx.xxx.xxx'
access_token = 'xxx.xxx.xxx.xxx'
access_secret = 'xxx.xxx.xxx.xxx'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler = auth,
                 parser = tweepy.parsers.JSONParser(),
                 wait_on_rate_limit = True, 
                 wait_on_rate_limit_notify = True)

In [8]:
def fetch_and_save(ids, api_ins, one_id = None):
    '''
    This function will fetch data with associated id in ids list
    ids (List Object): a list all tweets
    api_ins (Tweepy Object): api object instance, will be used to query twitter for data
    one_id (int): use when you want to query only for one tweet
    failed_ids (List Object): a list will be retured so that, this fuction can be called once again on those ids
    '''
    new_file_name = ''; failed_ids = []; tweet_df = []
    
    # checking if file exists
    if os.path.exists(files_list[2]):
        temp = [s for s in os.listdir() if "tweet_json" in s]
        new_file_name = files_list[2].split('.')[0] + "_" + str(len(temp)) + ".txt"
    else:
        new_file_name = files_list[2]
    
    # querying a list of ids
    if one_id == None:
        with open(new_file_name, mode = 'w') as outfile:
            for one_id in ids:
                try:
                    content = api_ins.get_status(one_id, tweet_mode='extended')
                    json.dump(content, outfile)
                    outfile.write('\n')
                
                except Exception as e:
                    print("Error for: " + str(one_id) + " - " + str(e))
                    failed_ids.append(one_id)
    
    # querying a single id
    else:
        try:
            content = api_ins.get_status(one_id, include_entities = True)
            favorites = content['favorite_count']
            retweets = content['retweet_count']
            
            tweet_df.append({'tweet_id': int(one_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets)})
            
            return tweet_df
                           
        except Exception as e:
            print("Error for: " + str(one_id) + " - " + str(e))
            failed_ids.append(one_id)

    return failed_ids

In [9]:
# passing the list of ids to the fuction fetch_and_save(), but in batches
# given that we can request 900 request/15min - window, let's break our ids into
tweet_ids = archive['tweet_id'].tolist()

# set_one, two and three
set_one = tweet_ids[0:900]; set_two = tweet_ids[900:1800]; set_three = tweet_ids[1800:]

# checking the lengths so that we send 900 ids/requests.
print(len(set_one), len(set_two), len(set_three), len(set_one)+len(set_two)+len(set_three))
print(len(set_one)+len(set_two)+len(set_three) == len(tweet_ids))

900 900 556 2356
True


In [10]:
# fetching data 1st iteration
# starting the timer
start = timer()

# querying
test_one = fetch_and_save(set_one, api)

# ending the timer
end = timer()

# calculating the runtime for fetch_and_save
print("That took about {} mins.".format(round((end - start)/60, 1)))

Error for: 888202515573088257 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 873697596434513921 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 872668790621863937 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 869988702071779329 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 866816280283807744 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 861769973181624320 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 845459076796616705 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 842892208864923648 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 837012587749474308 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 827228250799742977 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 802247111496568832 - [{'code': 144, 'message': 'No status found with 

In [11]:
# no of erroneous ids
print("we have about {} failed requests.".format(len(test_one)))

we have about 13 failed requests.


In [12]:
# fetching data 2nd iteration
# starting the timer
start = timer()

# querying
test_two = fetch_and_save(set_two, api)

# ending the timer
end = timer()

# calculating the runtime for fetch_and_save
print("That took about {} mins.".format(round((end - start)/60, 1)))

Rate limit reached. Sleeping for: 397


Error for: 758740312047005698 - Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))
Error for: 754011816964026368 - [{'code': 144, 'message': 'No status found with that ID.'}]
That took about 13.7 mins.


In [13]:
# no of erroneous ids
print("we have about {} failed requests.".format(len(test_two)))

we have about 2 failed requests.


In [14]:
# fetching data 3rd iteration
# starting the timer
start = timer()

# querying
test_three = fetch_and_save(set_three, api)

# ending the timer
end = timer()

# calculating the runtime for fetch_and_save
print("That took about {} mins.".format(round((end - start)/60, 1)))

Rate limit reached. Sleeping for: 316


Error for: 676957860086095872 - Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))
That took about 9.8 mins.


In [15]:
# no of erroneous ids
print("we have about {} failed requests.".format(len(test_three)))

we have about 1 failed requests.


In [24]:
# lets save the failed ids into one master list
failed_ids = test_one + test_two + test_three
print("Total failed request are: {}. \n".format(len(failed_ids)))

# ids that failed and the ones that passed
indi_fail = []; success = []

#for each failed id, lets try to fetch status individually.
for failed_id in failed_ids:
    temp = fetch_and_save(ids = None, api_ins = api, one_id = failed_id)
    indi_fail.append(temp[0])

# removing empty elements from list
success = [x for x in indi_fail if not isinstance(x, (int))]
indi_fail = [x for x in indi_fail if isinstance(x, (int))]

# checking if there is change
print("\nWe were able to retrieve {} records, others failed.".format(len(failed_ids) - len(indi_fail)))

Total failed request are: 16. 

Error for: 888202515573088257 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 873697596434513921 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 872668790621863937 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 869988702071779329 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 866816280283807744 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 861769973181624320 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 845459076796616705 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 842892208864923648 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 837012587749474308 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 827228250799742977 - [{'code': 144, 'message': 'No status found with that ID.'}]
Error for: 802247111496568832 - [{'code': 144, '

**<span style="color:red"> We have 14 missing records, the following were confirmed using `twurl` command from Ruby/Gems.</span>**<br>
**<span style="color:green"> The possible reason is.</span>**

![error](error.png)

4. [x] **Okay, let's combine the successful jsons into one file, called the `tweet_master.txt`**

In [26]:
# combing all successful jsons into one master file
json_1 = pd.read_json('tweet_json.txt', lines = True, encoding = 'utf-8')
json_2 = pd.read_json('tweet_json_1.txt', lines = True, encoding = 'utf-8')
json_3 = pd.read_json('tweet_json_2.txt', lines = True, encoding = 'utf-8')

# total rows that we need to have in our resulting dataframe
json_1.shape, json_2.shape, json_3.shape

((887, 32), (898, 32), (555, 28))

In [27]:
json_master = pd.concat([json_1, json_2, json_3], ignore_index = True, join = 'outer', sort = True)
json_master.to_json('tweet_master.txt', orient = 'records', lines = True)

In [28]:
# removing objects that are not required.
del archive, img_pre_test
del json_1, json_2, json_3, json_master
del indi_fail, end, start, test_one, test_two, test_three, set_one, set_two, set_three
del consumer_key, consumer_secret, access_token, access_secret, auth, api

# we are not removing success and files_list, making sure we stick to good programming practices - reusablity.

5. [x] **Last thing to do is to tidy up our folder, let's get going.**

In [29]:
# moving all data files under one folder - dataset
# removing the temporary files, that acted as placeholders

# creating the folder
folder = 'dataset'
if not os.path.exists(folder):
    os.mkdir(folder)

# we know that our master datasets for this project are
# 1. twitter-archive-enhanced.csv
# 2. image-predictions.tsv
# 3. tweet_json_master.txt
# let us move these files

# updating our files_list
files_list[-1] = 'tweet_master.txt'

# moving only required files
for file in files_list:
    if os.path.exists(file):
        os.rename(file, folder+'/'+file)

# removing the tweet_json and tweet_json_1 files as they are not required anymore
for file in [s for s in os.listdir() if "tweet_json" in s]:
    if os.path.exists(file):
        os.remove(file)
    
# lisitng the current directory
os.listdir()

# clean and neat, lets get with assessing and cleaning

['.git',
 '.ipynb_checkpoints',
 'dataset',
 'error.png',
 'New Text Document.txt',
 'README.md',
 'wrangle_act.ipynb']

In [30]:
# renaming files_list
for i in range(3):
    files_list[i] = folder + '/'+ files_list[i]

5. [x] **Last thing to do is to extract `retweet_count` and `favourite_count` from `tweet_master.txt`, saving the result as .csv**

In [35]:
tweet_json = pd.read_json(files_list[2], lines = True, encoding = 'utf-8')
tweet_json = tweet_json[['id', 'retweet_count','favorite_count']]
tweet_json.rename(index = str, columns={'id' : 'tweet_id', 'retweet_count': 'retweets','favorite_count': 'favorites'}, inplace = True)
tweet_json.append(success, ignore_index = True)
tweet_json.to_json(files_list[2], orient = 'records', lines = True)
tweet_json.sample(3)

,tweet_id,retweets,favorites
2211,668268907921326080,243,565
510,810254108431155201,3699,15766
1861,675109292475830276,1190,2884


## Assessing